In [11]:
%connect_info

{
  "hb_port": 41795,
  "kernel_name": "",
  "control_port": 44968,
  "stdin_port": 48559,
  "key": "f8de8d88-bb9f-4c52-a213-1bbb5c844397",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "iopub_port": 53253,
  "shell_port": 58502,
  "ip": "127.0.0.1"
}

Paste the above JSON into a file, and connect with:
    $> ipython <app> --existing <file>
or, if you are local, you can connect with just:
    $> ipython <app> --existing /run/user/1000/jupyter/kernel-43ca6bbb-8f9d-4858-a6a7-fdeed293c05b.json 
or even just:
    $> ipython <app> --existing 
if this is the most recent IPython session you have started.


In [2]:
import sys
sys.path.append( "/home/peer/PycharmProjects/neslearng")

In [3]:
import pandas
import hsa.ba.rewards as rewards
import itertools
from extern.fceux_learningenv.nes_python_interface.nes_python_interface import NESInterface
import hsa.machine_constants
from hsa.nes_python_input import py_to_nes_wrapper
from hsa.visualization.parse_fm2 import parse_fm2

In [4]:
movies = !ls ../../movies/*.*

In [5]:
reward_functions = {
    "scaled_for_time_left":lambda:rewards.scaled_for_time_left,
    "delta_potential":rewards.make_delta_potential,
    "main_reward":rewards.make_main_reward,
    "ehrenbrav":lambda:"ehrenbrav",
    "mario_x_speed":lambda:rewards.mario_x_speed,
    "delta_points":rewards.make_delta_points
}

In [6]:
stuff_to_check = list(itertools.product(movies,reward_functions.items()))

In [24]:
stuff_to_check

[('../../movies/10_1-1_slow_with_powerup.fm2',
  ('mario_x_speed', <function __main__.<lambda>>)),
 ('../../movies/10_1-1_slow_with_powerup.fm2',
  ('delta_potential', <function hsa.ba.rewards.make_delta_potential>)),
 ('../../movies/10_1-1_slow_with_powerup.fm2',
  ('delta_points', <function hsa.ba.rewards.make_delta_points>)),
 ('../../movies/10_1-1_slow_with_powerup.fm2',
  ('ehrenbrav', <function __main__.<lambda>>)),
 ('../../movies/10_1-1_slow_with_powerup.fm2',
  ('main_reward', <function hsa.ba.rewards.make_main_reward>)),
 ('../../movies/10_1-1_slow_with_powerup.fm2',
  ('scaled_for_time_left', <function __main__.<lambda>>)),
 ('../../movies/11_1-1_small_walking.fm2',
  ('mario_x_speed', <function __main__.<lambda>>)),
 ('../../movies/11_1-1_small_walking.fm2',
  ('delta_potential', <function hsa.ba.rewards.make_delta_potential>)),
 ('../../movies/11_1-1_small_walking.fm2',
  ('delta_points', <function hsa.ba.rewards.make_delta_points>)),
 ('../../movies/11_1-1_small_walking.f

In [12]:
def reward_from_movie_with_func(movie_path, reward_function_factory):
    reward_function_instance = reward_function_factory()
    nes = NESInterface(hsa.machine_constants.mario_rom_location, eb_compatible=False, auto_render_period=3, reward_function=reward_function_instance)
    with open(movie_path) as movie_file:
        reward_sum = 0
        for combi in parse_fm2(movie_file):
            reward = nes.act(py_to_nes_wrapper(combi))
            reward_sum += reward
    del nes
    return reward_sum


In [13]:
reward_from_movie_with_func(movies[1],reward_functions["ehrenbrav"])

283080650

In [14]:
reward_from_movie_with_func(movies[1],reward_functions["mario_x_speed"])

1079.0999999999894

In [ ]:
score_tuples = []
for movie, (reward_name, reward_factory) in stuff_to_check:
    score_tuples.append((movie,reward_name,reward_from_movie_with_func(movie,reward_factory)))

In [8]:
pandas.DataFrame(data=score_tuples,columns=["movie","reward_name","sum"])

,movie,reward_name,sum
0,../../movies/10_1-1_slow_with_powerup.fm2,scaled_for_time_left,5051
1,../../movies/10_1-1_slow_with_powerup.fm2,mario_x_speed,5051
2,../../movies/10_1-1_slow_with_powerup.fm2,delta_potential,5051
3,../../movies/10_1-1_slow_with_powerup.fm2,delta_points,5051
4,../../movies/10_1-1_slow_with_powerup.fm2,main_reward,5051
5,../../movies/10_1-1_slow_with_powerup.fm2,ehrenbrav,5051
6,../../movies/11_1-1_small_walking.fm2,scaled_for_time_left,3595
7,../../movies/11_1-1_small_walking.fm2,mario_x_speed,3595
8,../../movies/11_1-1_small_walking.fm2,delta_potential,3595
9,../../movies/11_1-1_small_walking.fm2,delta_points,3595
